In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

#os.environ["SM_FRAMEWORK"] = "tf.keras"

In [2]:
from tools.config import setup
from treeseg.data.dataset import prepare_datasets
from treeseg.modeling.trainer import trainer
from treeseg.modeling.builder import resume_or_load
from treeseg.evaluation.evaluator import evaluator
from treeseg.utils.callbacks import build_callbacks

/Users/anisr/Documents/TreeSeg/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
def run(conf, eval_only):
    assert os.path.exists(
        conf.data_folder
    ), f"Data folder {conf.data_folder} does not exist."

    if not os.path.exists(conf.output_dir):
            os.makedirs(conf.output_dir)

    train_dataset, val_dataset, test_dataset = prepare_datasets(conf.data_folder, conf)

    model, optimizer, criterion, metrics = resume_or_load(conf)
    
    n_batches = len(train_dataset)

    callbacks = build_callbacks(n_batches, conf.output_dir, optimizer)

    if eval_only:
        print("Evaluation-only mode started.")

        evaluator(model, test_dataset, len(test_dataset), conf.test_batch_size, conf.threshold)

    else:
        print("Training mode started.")
        
        trainer(model, optimizer, criterion, metrics, train_dataset, val_dataset, conf, callbacks)


if __name__ == "__main__":
    config_file_path = "../configs/unet_self_attention_bs8_cs256.txt"

    conf = setup(config_file_path)

    # Modified Config Variables for Local Execution
    conf.data_folder = "/Users/anisr/Documents/AerialImageModel_ITD"
    conf.output_dir = os.path.join("..", conf.output_dir)

    eval_only = False

    run(conf, eval_only)

No checkpoint found. Proceeding without loading weights.
Training mode started.


KeyboardInterrupt: 